In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [193]:
nQuizz=11
sQuizz="08"
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
# fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/24-Quizz/Quizz-%s.csv"%sQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")
print fCM,fQuizzoodle

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv /Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/24-Quizz/Quizz-08.csv


In [194]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [195]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
# alias

In [196]:
len(alias)

393

In [197]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [198]:
lQuizz, dfCM

(                          pseudo  points   temps
 0                   lisa-cremers       5  108.02
 1                            Tic       4   79.48
 2                    lea-cremers       4   86.40
 3          Sarah.millepiedtorres       4  109.15
 4         Georges--Jeanne_Anaële       3   97.78
 5       Raffy M Tovo L Lacoume J       3  109.88
 6                   frida-ocampo       3  113.78
 7                   aurices.elya       3  124.91
 8                  adele-sombrun       3  129.18
 9                   agatheraynal       3  129.55
 10              Le Peillet-Aloys       3  131.24
 11          marina-daniel-ivanov       3  134.94
 12               MARCENAT Marine       3  144.03
 13                Méchain Hermès       2   68.42
 14                         Paola       2   76.49
 15                  julie-pouget       2   79.48
 16           a-montoya/ a-salomé       2   83.29
 17                 Ferrol-maelys       2   87.31
 18           Marixtu-D gabriel-A       2   87.45


In [199]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Tic
a-montoya/ a-salomé
Marixtu-D gabriel-A
Amandine.berloin
mélissa-bonneau, maelle-trou
Flavie guilcher
Laura-lecomte
Laura Pourieux
Flavie


# Listage des alias

In [200]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P-N"]=dfCM[u"prénom"].str.lower().str.strip()+"-"+dfCM[u"nom"].str.lower().str.strip()
dfCM["N-P"]=dfCM[u"nom"].str.lower().str.strip()+"-"+dfCM[u"prénom"].str.lower().str.strip()
dfCM["P.N"]=dfCM[u"prénom"].str.lower().str.strip()+"."+dfCM[u"nom"].str.lower().str.strip()
dfCM["N.P"]=dfCM[u"nom"].str.lower().str.strip()+"."+dfCM[u"prénom"].str.lower().str.strip()
# dfCM

In [201]:
pn=dfCM["PN"].tolist()
pnT=dfCM["P-N"].tolist()
pnD=dfCM["P.N"].tolist()
np=dfCM["NP"].tolist()
npT=dfCM["N-P"].tolist()
npD=dfCM["N.P"].tolist()
lNomPrenom={"PN":pn,"P-N":pnT,"P.N":pnD,"NP":np,"N-P":npT,"N.P":npD}

In [202]:
for pseudo in pseudos:
    for nomPrenom in lNomPrenom:
        if unidecode.unidecode(pseudo.lower()) in lNomPrenom[nomPrenom]:
            print pseudo,dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
            alias[pseudo]=[str(dfCM[dfCM[nomPrenom]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

Amandine.berloin 429252
Flavie guilcher 435376
Laura-lecomte 436164
Laura Pourieux 437937


In [203]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Tic
a-montoya/ a-salomé
Marixtu-D gabriel-A
mélissa-bonneau, maelle-trou
Flavie


In [204]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

Tic


Nom a chercher:
 


a-montoya/ a-salome


Nom a chercher:
 montoy


              PN      id
106  ana montoya  438210


...

id :
 439214 435953


Flavie


Nom a chercher:
 flavie


                 PN      id
25   flavie brouxel  437820
67  flavie guilcher  435376


id :
 435376


In [205]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [206]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [207]:
pseudos

[u'Tic',
 u'a-montoya/ a-salom\xe9',
 u'Marixtu-D gabriel-A',
 u'm\xe9lissa-bonneau, maelle-trou',
 u'Flavie']

In [208]:
# alias["Sofiane"]

In [209]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [210]:
len(alias)

401

In [211]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {u'A Salom\xe9 - a Montoya': ['438210', '438330'],
  u'A.salom\xe9 a.montoya': ['438210', '438330'],
  'APPRIOU': ['437791'],
  'APPRIOU-Maurine': ['437791'],
  'AUDLUU': ['435643'],
  'Aberard Florelle': ['440340'],
  'Aberard_Florelle': ['440340'],
  'Adam-eulalie': ['435916'],
  'Adriana Mangiarratti': ['442806'],
  'Adriana-Mangiarratti': ['442806'],
  'Agathe raynal': ['436720'],
  'Agatheraynal': ['436720'],
  'Akpinar Neslihan': ['438199'],
  u'Amandine.berloin': ['429252'],
  'Amelia Collins': ['437856'],
  'Angelina': ['438425'],
  'Angelina_Boukono': ['438425'],
  'Assimay nahi': ['436153'],
  'Audrey-luu': ['435643'],
  'Axelle-Meteau, Anais-Vieira': ['437803', '437028'],
  'BONNEAU MELISSA': ['439214'],
  'BONNEAU Melissa': ['439214'],
  u'BONNEAU-M\xe9lissa,TROUILLET-M': ['439214', '435953'],
  'Baldy Camille': ['436384'],
  'Benhamouda-souha': ['442845'],
  'Boileau-Valentine': ['440209'],
  'Bordelait Lizzie': ['439788'],
  'Bordelait-Lizzie': ['439788'],
 

In [212]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [213]:
nbQuestions=[7,7,7,7,7,7,7,6,6,6,6]
nbQ=float(nbQuestions[nQuizz-1])
print nbQ

6.0


In [214]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [215]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,lisa-cremers,5,108.02,[440801],18,440801,None,None
1,Tic,4,79.48,None,17,None,None,None
2,lea-cremers,4,86.40,[440791],17,440791,None,None
3,Sarah.millepiedtorres,4,109.15,[437850],17,437850,None,None
4,Georges--Jeanne_Anaële,3,97.78,[436330],15,436330,None,None
5,Raffy M Tovo L Lacoume J,3,109.88,"[441372, 438084, 437894]",15,441372,438084,437894
6,frida-ocampo,3,113.78,[441958],15,441958,None,None
7,aurices.elya,3,124.91,[440486],15,440486,None,None
8,adele-sombrun,3,129.18,[441732],15,441732,None,None
9,agatheraynal,3,129.55,[436720],15,436720,None,None


In [216]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [224]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True]).reset_index(drop=True)
df123

,pseudo,note,temps,id
0,lisa-cremers,18,108.02,440801
1,lea-cremers,17,86.40,440791
2,Sarah.millepiedtorres,17,109.15,437850
3,Georges--Jeanne_Anaële,15,97.78,436330
4,Raffy M Tovo L Lacoume J,15,109.88,441372
5,Raffy M Tovo L Lacoume J,15,109.88,438084
6,Raffy M Tovo L Lacoume J,15,109.88,437894
7,frida-ocampo,15,113.78,441958
8,aurices.elya,15,124.91,440486
9,adele-sombrun,15,129.18,441732


In [225]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [226]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"].any()<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,lisa-cremers,20,108.02,440801
1,lea-cremers,18,86.40,440791
2,Sarah.millepiedtorres,17,109.15,437850
3,Georges--Jeanne_Anaële,15,97.78,436330
4,Raffy M Tovo L Lacoume J,15,109.88,441372
5,Raffy M Tovo L Lacoume J,15,109.88,438084
6,Raffy M Tovo L Lacoume J,15,109.88,437894
7,frida-ocampo,15,113.78,441958
8,aurices.elya,15,124.91,440486
9,adele-sombrun,15,129.18,441732


In [227]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")

In [228]:
dfNotes

,pseudo,note,temps
id,,,
440801,lisa-cremers,20,108.02
440791,lea-cremers,18,86.40
437850,Sarah.millepiedtorres,17,109.15
440486,aurices.elya,15,124.91
437984,MARCENAT Marine,15,144.03
437894,Raffy M Tovo L Lacoume J,15,109.88
436720,agatheraynal,15,129.55
436648,marina-daniel-ivanov,15,134.94
436330,Georges--Jeanne_Anaële,15,97.78
